## Imports and loading data

In [1]:
import tensorflow as tf
import numpy as np

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
image_size=(256,256)
dir="data/resized/"
split=0.1
dataset=tf.keras.preprocessing.image_dataset_from_directory(dir+"train",batch_size=32,image_size=image_size,
                                                            validation_split=split,
                                                            subset="training",
                                                            seed=123,
                                                            shuffle=True,color_mode="rgb")
test_dataset=tf.keras.preprocessing.image_dataset_from_directory(dir+"train",batch_size=32,image_size=image_size,
                                                            validation_split=split,
                                                            seed=123,
                                                            subset="validation",
                                                            shuffle=True,color_mode="rgb")



Found 10868 files belonging to 9 classes.
Using 9782 files for training.
Found 10868 files belonging to 9 classes.
Using 1086 files for validation.


## Loading VGG16 model

In [3]:
b=dataset.take(1)
for i,l in b:
    print(l)

tf.Tensor([0 3 3 3 6 3 2 0 0 2 7 2 8 5 7 3 2 5 8 3 5 4 2 8 0 2 3 0 7 4 5 2], shape=(32,), dtype=int32)


In [4]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

#????base_model=InceptionV3(weights="imagenet",include_top=False,input_shape=(256,256,3))
#base_model = VGG16(weights="imagenet", include_top=False, input_shape=(256,256,3))
#base_model = ResNet101(weights="imagenet", include_top=False, input_shape=(256,256,3))
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(256,256,3))
#base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(256,256,3))
model_name="ResNet50"
base_model.trainable = False


In [5]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

## Adding Layers

In [6]:
from tensorflow.keras import layers, models


flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(1024, activation='relu')
#dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(9, activation='softmax')



model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
 #   dense_layer_2,
    prediction_layer
])


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              134218752 
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 9225      
Total params: 157,815,689
Trainable params: 134,227,977
Non-trainable params: 23,587,712
_________________________________________________________________


## Training model

In [8]:
from tensorflow.keras.callbacks import EarlyStopping,CSVLogger

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
csvlogger=CSVLogger("log-"+model_name+"-"+str(split)+".csv")
history=model.fit(dataset, validation_data=test_dataset,epochs=15, batch_size=32, callbacks=[es,csvlogger])

Epoch 1/15
306/306 [==============================] - 70s 209ms/step - loss: 7.1594 - accuracy: 0.8837 - val_loss: 0.5163 - val_accuracy: 0.9337
Epoch 2/15
306/306 [==============================] - 63s 207ms/step - loss: 0.2375 - accuracy: 0.9692 - val_loss: 0.2248 - val_accuracy: 0.9705
Epoch 3/15
306/306 [==============================] - 66s 214ms/step - loss: 0.0848 - accuracy: 0.9849 - val_loss: 0.4450 - val_accuracy: 0.9678
Epoch 4/15
306/306 [==============================] - 68s 222ms/step - loss: 0.1955 - accuracy: 0.9844 - val_loss: 0.7722 - val_accuracy: 0.9457
Epoch 5/15
306/306 [==============================] - 71s 231ms/step - loss: 0.2949 - accuracy: 0.9764 - val_loss: 0.5036 - val_accuracy: 0.9715
Epoch 6/15
306/306 [==============================] - 73s 239ms/step - loss: 0.3574 - accuracy: 0.9821 - val_loss: 0.4786 - val_accuracy: 0.9742
Epoch 7/15
306/306 [==============================] - 78s 255ms/step - loss: 0.2285 - accuracy: 0.9862 - val_loss: 0.4831 - val_ac

In [ ]:
print(history.history.keys())
print(history.history['val_loss'])
model.evaluate(test_dataset)

In [ ]:
imgs=[]
labels=[]
test_dataset=test_dataset.unbatch()
for x,y in test_dataset.as_numpy_iterator(): 
    imgs.append(x)
    labels.append(y)
imgs=np.array(imgs)
labels=np.array(labels)

In [ ]:
probabilities=model.predict(imgs)
predictions=np.argmax(probabilities,axis=1)


In [ ]:
m=tf.math.confusion_matrix(labels,predictions)

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 7))
sb.heatmap(m/np.sum(m,axis=1).reshape(9,1), xticklabels=dataset.class_names, yticklabels=dataset.class_names, 
            annot=True,fmt=".2f")
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

In [ ]:
base_model.trainable=True
with tf.device('/cpu:0'):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )
    history=model.fit(dataset, validation_data=test_dataset,epochs=2, batch_size=32)